In [4]:
import nfl_data_py as nfl

import pandas as pd
import numpy as np

# --------------------------------------------------------------
# Day 19 : NFL.py 🏈
# --------------------------------------------------------------

# EDA and objectives

### Goal: Build a binary classifier to determine if a first round quarterback is going to be a bust

### Today: Build a training dataset

In [31]:
years = range(2010, 2023)

In [32]:
data = nfl.import_weekly_data(years)
draft = nfl.import_draft_picks(years)

Downcasting floats.


In [168]:
data

player_id player_name player_display_name position position_group  \
0     00-0000865        None       Charlie Batch       QB             QB   
1     00-0000865        None       Charlie Batch       QB             QB   
2     00-0000865        None       Charlie Batch       QB             QB   
3     00-0001231        None        Josh Bidwell        P           SPEC   
4     00-0001545        None         Todd Bouman       QB             QB   
...          ...         ...                 ...      ...            ...   
5626  00-0038134    K.Walker      Kenneth Walker       RB             RB   
5627  00-0038134    K.Walker      Kenneth Walker       RB             RB   
5628  00-0038134    K.Walker      Kenneth Walker       RB             RB   
5629  00-0038147   D.Baldwin      Daylen Baldwin       WR             WR   
5630  00-0038149       J.Ali            Josh Ali       WR             WR   

                                           headshot_url recent_team  season  \
0                                                  None         PIT    2010   
1                                                  None         PIT    2010   
2                                                  None         PIT    2010   
3                                                  None         WAS    2010   
4                                                  None         JAX    2010   
...                                                 ...         ...     ...   
5626  https://static.www.nfl.com/image/private/f_aut...         SEA    2022   
5627  https://static.www.nfl.com/image/private/f_aut...         SEA    2022   
5628  https://static.www.nfl.com/image/private/f_aut...         SEA    2022   
5629  https://static.www.nfl.com/image/private/f_aut...         CLE    2022   
5630  https://static.www.nfl.com/image/private/f_aut...         ATL    2022   

      week season_type  ... receiving_first_downs  receiving_epa  \
0        2         REG  ...                   0.0            NaN   
1        3         REG  ...                   0.0            NaN   
2        4         REG  ...                   0.0            NaN   
3        1         REG  ...                   0.0            NaN   
4        7         REG  ...                   0.0            NaN   
...    ...         ...  ...                   ...            ...   
5626    17         REG  ...                   1.0       0.968106   
5627    18         REG  ...                   0.0       0.430252   
5628    19        POST  ...                   0.0      -0.309929   
5629    15         REG  ...                   2.0       1.791362   
5630    18         REG  ...                   0.0      -0.519020   

      receiving_2pt_conversions  racr  target_share  air_yards_share  \
0                             0   NaN           NaN              NaN   
1                             0   NaN           NaN              NaN   
2                             0   NaN           NaN              NaN   
3                             0   NaN           NaN              NaN   
4                             0   NaN           NaN              NaN   
...                         ...   ...           ...              ...   
5626                          0  -9.0      0.035714        -0.003717   
5627                          0 -10.0      0.035714        -0.003745   
5628                          0   0.0      0.029412         0.000000   
5629                          0   1.0      0.076923         0.223214   
5630                          0   0.0      0.035714         0.134731   

          wopr  special_teams_tds  fantasy_points  fantasy_points_ppr  
0          NaN                0.0        1.000000            1.000000  
1          NaN                0.0       18.040001           18.040001  
2          NaN                0.0        4.040000            4.040000  
3          NaN                0.0        0.000000            0.000000  
4          NaN                0.0       13.680000           13.680000  
...        ...                .

In [169]:
draft

season  round  pick team     gsis_id pfr_player_id     cfb_player_id  \
8577     2010      1     1  STL  00-0027854      BradSa00    sam-bradford-1   
8578     2010      1     2  DET  00-0027855      SuhxNd99   ndamukong-suh-1   
8579     2010      1     3  TAM  00-0027856      McCoGe99    gerald-mccoy-1   
8580     2010      1     4  WAS        None      WillTr21  trent-williams-2   
8581     2010      1     5  KAN  00-0027858      BerrEr99      eric-berry-1   
...       ...    ...   ...  ...         ...           ...               ...   
11892    2017      7   249  SEA  00-0033594      CarsCh00    chris-carson-1   
11893    2017      7   250  DET  00-0033595      OConPa00     pat-oconnor-1   
11894    2017      7   251  CIN  00-0033798      SchrMa00   mason-schreck-1   
11895    2017      7   252  CLE  00-0033596      DayeMa00      matt-dayes-1   
11896    2017      7   253  DEN  00-0033597      KellCh00      chad-kelly-1   

      pfr_player_name    hof position  ... pass_ints rush_atts rush_yards  \
8577     Sam Bradford  False       QB  ...      61.0     146.0      340.0   
8578    Ndamukong Suh  False       DT  ...       0.0       0.0        0.0   
8579     Gerald McCoy  False       DT  ...       0.0       0.0        0.0   
8580   Trent Williams  False        T  ...       0.0       0.0        0.0   
8581       Eric Berry  False       DB  ...       0.0       0.0        0.0   
...               ...    ...      ...  ...       ...       ...        ...   
11892    Chris Carson  False       RB  ...       0.0     769.0     3502.0   
11893    Pat O'Connor  False       DE  ...       0.0       0.0        0.0   
11894   Mason Schreck  False       TE  ...       0.0       0.0        0.0   
11895   Matthew Dayes  False       RB  ...       0.0       5.0       13.0   
11896      Chad Kelly  False       QB  ...       0.0       1.0       -1.0   

       rush_tds  receptions  rec_yards  rec_tds  def_solo_tackles  def_ints  \
8577        2.0         1.0        5.0      0.0               NaN       NaN   
8578        0.0         0.0        0.0      0.0             392.0       1.0   
8579        0.0         0.0        0.0      0.0             235.0       NaN   
8580        0.0         0.0        0.0      0.0               1.0       NaN   
8581        0.0         0.0        0.0      0.0             377.0      14.0   
...         ...         ...        ...      ...               ...       ...   
11892      24.0       107.0      804.0      7.0               NaN       NaN   
11893       0.0         0.0        0.0      0.0              14.0       NaN   
11894       0.0         2.0        6.0      0.0               1.0       NaN   
11895       0.0         4.0       29.0      0.0               9.0       NaN   
11896       0.0         0.0        0.0      0.0               NaN       NaN   

      def_sacks  
8577        NaN  
8578       71.5  
8579       59.5  
8580        NaN  
8581        5.5  
...         ...  
11892       NaN  
11893       1.5  
11894       NaN  
11895       NaN  
11896       NaN  

[3320 rows x 36 columns]

In [78]:
df = data[(data['season'] >= 2010) & (data['position'] == 'QB')]

In [61]:
df = df.groupby('player_display_name').agg(
    {'player_id':pd.Series.mode, 'position':pd.Series.mode, 'season':'min', 'recent_team':lambda x:x.iloc[0]}
    ).reset_index().sort_values('player_display_name')
rounds = [draft[draft['pfr_player_name'] == plyr]['round'] for plyr in df['player_display_name']]

In [118]:
df_draft = draft[(draft['round'] == 1) & (draft['position'] == 'QB') & ()][['pfr_player_name', 'season', 'round', 'pick', 'team']]
df_draft.loc[df_draft['team'] == 'STL', 'team'] = 'LA'

In [158]:
players_by_season[players_by_season['player_display_name'] == 'Patrick Mahomes']

player_display_name  season recent_team
744     Patrick Mahomes    2017          KC
745     Patrick Mahomes    2018          KC
746     Patrick Mahomes    2019          KC
747     Patrick Mahomes    2020          KC
748     Patrick Mahomes    2021          KC
749     Patrick Mahomes    2022          KC

In [ ]:
labels = [df[df['player_display_name'] == plyr] for plyr in df_draft['pfr_player_name']]

In [119]:
players_by_season = df.groupby(['player_display_name', 'season']).agg({'recent_team':lambda x: x.iloc[0]}).reset_index()

In [120]:
players_by_season[players_by_season['player_display_name'] == 'Sam Bradford']

player_display_name  season recent_team
828        Sam Bradford    2010          LA
829        Sam Bradford    2011          LA
830        Sam Bradford    2012          LA
831        Sam Bradford    2013          LA
832        Sam Bradford    2015         PHI
833        Sam Bradford    2016         MIN
834        Sam Bradford    2017         MIN
835        Sam Bradford    2018         ARI

In [127]:
players_by_season[(players_by_season['player_display_name'] == plyr) & (players_by_season['recent_team'].values[0] == draft_team)].shape[0]

8

In [ ]:
df[(df['ITEM'])]

In [135]:
df_draft['pfr_player_name']

8577           Sam Bradford
8601              Tim Tebow
8832             Cam Newton
8839            Jake Locker
8841         Blaine Gabbert
8843       Christian Ponder
9086            Andrew Luck
9087     Robert Griffin III
9093         Ryan Tannehill
9107         Brandon Weeden
9354              EJ Manuel
9595          Blake Bortles
9614         Johnny Manziel
9624      Teddy Bridgewater
9849         Jameis Winston
9850         Marcus Mariota
10105            Jared Goff
10106          Carson Wentz
10130          Paxton Lynch
10377         Kenny Pickett
10620       Trevor Lawrence
10621           Zach Wilson
10622            Trey Lance
10630         Justin Fields
10634             Mac Jones
10879            Joe Burrow
10883        Tua Tagovailoa
10884        Justin Herbert
10904           Jordan Love
11134          Kyler Murray
11139          Daniel Jones
11148        Dwayne Haskins
11388        Baker Mayfield
11390           Sam Darnold
11394            Josh Allen
11397            Jos

In [136]:
players_by_season[players_by_season['player_display_name'] == 'Cam Newton']

player_display_name  season recent_team
153          Cam Newton    2011         CAR
154          Cam Newton    2012         CAR
155          Cam Newton    2013         CAR
156          Cam Newton    2014         CAR
157          Cam Newton    2015         CAR
158          Cam Newton    2016         CAR
159          Cam Newton    2017         CAR
160          Cam Newton    2018         CAR
161          Cam Newton    2019         CAR
162          Cam Newton    2020          NE
163          Cam Newton    2021         CAR

In [160]:
players_by_season[(players_by_season['player_display_name'] == 'Patrick Mahomes') & (players_by_season['recent_team'] == 'KC')].shape[0]

6

In [152]:
df_draft[df_draft['pfr_player_name'] == plyr]['season']

8577    2010
Name: season, dtype: int32

In [163]:
df_draft[df_draft['pfr_player_name'] == 'Patrick Mahomes']['team']

11653    KAN
Name: team, dtype: object

In [165]:
df_draft[df_draft['team'] == 'KAN']

pfr_player_name  season  round  pick team
11653  Patrick Mahomes    2017      1    10  KAN

In [161]:
labels = []
for plyr in df_draft['pfr_player_name']:
    draft_team = np.squeeze(df_draft[df_draft['pfr_player_name'] == plyr]['team'])
    draft_year = np.squeeze(df_draft[df_draft['pfr_player_name'] ==plyr]['season'].values[0])
    labels.append([plyr,players_by_season[(players_by_season['player_display_name'] == plyr) & (players_by_season['recent_team'] == draft_team)].shape[0] >= 5])

In [167]:
labels[:10]

[['Sam Bradford', False],
 ['Tim Tebow', False],
 ['Cam Newton', True],
 ['Jake Locker', False],
 ['Blaine Gabbert', False],
 ['Christian Ponder', False],
 ['Andrew Luck', True],
 ['Robert Griffin III', False],
 ['Ryan Tannehill', True],
 ['Brandon Weeden', False]]

player_id player_name player_display_name position position_group  \
0     00-0000865        None       Charlie Batch       QB             QB   
1     00-0000865        None       Charlie Batch       QB             QB   
2     00-0000865        None       Charlie Batch       QB             QB   
4     00-0001545        None         Todd Bouman       QB             QB   
5     00-0002110        None        Mark Brunell       QB             QB   
...          ...         ...                 ...      ...            ...   
5593  00-0038128    M.Willis        Malik Willis       QB             QB   
5594  00-0038128    M.Willis        Malik Willis       QB             QB   
5595  00-0038128    M.Willis        Malik Willis       QB             QB   
5596  00-0038128    M.Willis        Malik Willis       QB             QB   
5597  00-0038128    M.Willis        Malik Willis       QB             QB   

                                           headshot_url recent_team  season  \
0                                                  None         PIT    2010   
1                                                  None         PIT    2010   
2                                                  None         PIT    2010   
4                                                  None         JAX    2010   
5                                                  None         NYJ    2010   
...                                                 ...         ...     ...   
5593  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5594  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5595  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5596  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5597  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   

      week season_type  ... receiving_first_downs  receiving_epa  \
0        2         REG  ...                   0.0            NaN   
1        3         REG  ...                   0.0            NaN   
2        4         REG  ...                   0.0            NaN   
4        7         REG  ...                   0.0            NaN   
5        4         REG  ...                   0.0            NaN   
...    ...         ...  ...                   ...            ...   
5593     9         REG  ...                   0.0            NaN   
5594    10         REG  ...                   0.0            NaN   
5595    13         REG  ...                   0.0            NaN   
5596    15         REG  ...                   0.0            NaN   
5597    16         REG  ...                   0.0            NaN   

      receiving_2pt_conversions  racr  target_share  air_yards_share  wopr  \
0                             0   NaN           NaN              NaN   NaN   
1                             0   NaN           NaN              NaN   NaN   
2                             0   NaN           NaN              NaN   NaN   
4                             0   NaN           NaN              NaN   NaN   
5                             0   NaN           NaN              NaN   NaN   
...                         ...   ...           ...              ...   ...   
5593                          0   NaN           NaN              NaN   NaN   
5594                          0   NaN           NaN              NaN   NaN   
5595                          0   NaN           NaN              NaN   NaN   
5596                          0   NaN           NaN              NaN   NaN   
5597                          0   NaN           NaN              NaN   NaN   

      special_teams_tds  fantasy_points  fantasy_points_ppr  
0                   0.0        1.000000            1.000000  
1                   0.0       18.040001           18.040001  
2                   0.0        4.040000            4.040000  
4                   0.0       13.680000           13.680000  
5                   0.0        0.280000            0.280000  
...                

In [66]:
draft[draft['pfr_player_name'] == 'Will Grier']

season  round  pick team     gsis_id pfr_player_id cfb_player_id  \
11233    2019      3   100  CAR  00-0035251      GrieWi00  will-grier-1   

      pfr_player_name    hof position  ... pass_ints rush_atts rush_yards  \
11233      Will Grier  False       QB  ...       4.0       7.0       22.0   

       rush_tds  receptions  rec_yards  rec_tds  def_solo_tackles  def_ints  \
11233       0.0         0.0        0.0      0.0               NaN       NaN   

      def_sacks  
11233       NaN  

[1 rows x 36 columns]

In [63]:
df

player_display_name   player_id position  season recent_team
0           A.J. Feeley  00-0020305       QB    2011          LA
1           AJ McCarron  00-0031288       QB    2015         CIN
2         Aaron Rodgers  00-0023459       QB    2010          GB
3            Alex Smith  00-0023436       QB    2010          SF
4           Alex Tanney  00-0029623       QB    2015         TEN
..                  ...         ...      ...     ...         ...
228        Tyrod Taylor  00-0028118       QB    2011         BAL
229         Vince Young  00-0024218       QB    2010         TEN
230          Will Grier  00-0035251       QB    2019         CAR
231   Zach Mettenberger  00-0031266       QB    2014         TEN
232         Zach Wilson  00-0037013       QB    2021         NYJ

[233 rows x 5 columns]

In [62]:
rounds

[Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 9086    1
 Name: round, dtype: int32,
 8866    2
 Name: round, dtype: int32,
 10293    6
 Name: round, dtype: int32,
 Series([], Name: round, dtype: int32),
 10494    4
 Name: round, dtype: int32,
 11388    1
 Name: round, dtype: int32,
 11109    7
 Name: round, dtype: int32,
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 8841    1
 Name: round, dtype: int32,
 9595    1
 Name: round, dtype: int32,
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 10305    6
 Name: round, dtype: int32,
 9107    1
 Name: round, dtype: int32,
 Series([], Name: round, dtype: int32),
 9995    5
 Name: round, dtype: int32,
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 Series([], Name: round, dtype: int32),
 Serie

In [ ]:
data

In [58]:
df

player_id position  season recent_team
player_display_name                                         
A.J. Feeley          00-0020305       QB    2011          LA
AJ McCarron          00-0031288       QB    2015         CIN
Aaron Rodgers        00-0023459       QB    2010          GB
Alex Smith           00-0023436       QB    2010          SF
Alex Tanney          00-0029623       QB    2015         TEN
...                         ...      ...     ...         ...
Tyrod Taylor         00-0028118       QB    2011         BAL
Vince Young          00-0024218       QB    2010         TEN
Will Grier           00-0035251       QB    2019         CAR
Zach Mettenberger    00-0031266       QB    2014         TEN
Zach Wilson          00-0037013       QB    2021         NYJ

[233 rows x 4 columns]

In [ ]:
def get_draft_round(draft_df, qb):
    round = draft

In [59]:
draft

season  round  pick team     gsis_id pfr_player_id     cfb_player_id  \
8577     2010      1     1  STL  00-0027854      BradSa00    sam-bradford-1   
8578     2010      1     2  DET  00-0027855      SuhxNd99   ndamukong-suh-1   
8579     2010      1     3  TAM  00-0027856      McCoGe99    gerald-mccoy-1   
8580     2010      1     4  WAS        None      WillTr21  trent-williams-2   
8581     2010      1     5  KAN  00-0027858      BerrEr99      eric-berry-1   
...       ...    ...   ...  ...         ...           ...               ...   
11892    2017      7   249  SEA  00-0033594      CarsCh00    chris-carson-1   
11893    2017      7   250  DET  00-0033595      OConPa00     pat-oconnor-1   
11894    2017      7   251  CIN  00-0033798      SchrMa00   mason-schreck-1   
11895    2017      7   252  CLE  00-0033596      DayeMa00      matt-dayes-1   
11896    2017      7   253  DEN  00-0033597      KellCh00      chad-kelly-1   

      pfr_player_name    hof position  ... pass_ints rush_atts rush_yards  \
8577     Sam Bradford  False       QB  ...      61.0     146.0      340.0   
8578    Ndamukong Suh  False       DT  ...       0.0       0.0        0.0   
8579     Gerald McCoy  False       DT  ...       0.0       0.0        0.0   
8580   Trent Williams  False        T  ...       0.0       0.0        0.0   
8581       Eric Berry  False       DB  ...       0.0       0.0        0.0   
...               ...    ...      ...  ...       ...       ...        ...   
11892    Chris Carson  False       RB  ...       0.0     769.0     3502.0   
11893    Pat O'Connor  False       DE  ...       0.0       0.0        0.0   
11894   Mason Schreck  False       TE  ...       0.0       0.0        0.0   
11895   Matthew Dayes  False       RB  ...       0.0       5.0       13.0   
11896      Chad Kelly  False       QB  ...       0.0       1.0       -1.0   

       rush_tds  receptions  rec_yards  rec_tds  def_solo_tackles  def_ints  \
8577        2.0         1.0        5.0      0.0               NaN       NaN   
8578        0.0         0.0        0.0      0.0             392.0       1.0   
8579        0.0         0.0        0.0      0.0             235.0       NaN   
8580        0.0         0.0        0.0      0.0               1.0       NaN   
8581        0.0         0.0        0.0      0.0             377.0      14.0   
...         ...         ...        ...      ...               ...       ...   
11892      24.0       107.0      804.0      7.0               NaN       NaN   
11893       0.0         0.0        0.0      0.0              14.0       NaN   
11894       0.0         2.0        6.0      0.0               1.0       NaN   
11895       0.0         4.0       29.0      0.0               9.0       NaN   
11896       0.0         0.0        0.0      0.0               NaN       NaN   

      def_sacks  
8577        NaN  
8578       71.5  
8579       59.5  
8580        NaN  
8581        5.5  
...         ...  
11892       NaN  
11893       1.5  
11894       NaN  
11895       NaN  
11896       NaN  

[3320 rows x 36 columns]

In [40]:
df

player_id player_name player_display_name position position_group  \
0     00-0000865        None       Charlie Batch       QB             QB   
1     00-0000865        None       Charlie Batch       QB             QB   
2     00-0000865        None       Charlie Batch       QB             QB   
4     00-0001545        None         Todd Bouman       QB             QB   
5     00-0002110        None        Mark Brunell       QB             QB   
...          ...         ...                 ...      ...            ...   
5593  00-0038128    M.Willis        Malik Willis       QB             QB   
5594  00-0038128    M.Willis        Malik Willis       QB             QB   
5595  00-0038128    M.Willis        Malik Willis       QB             QB   
5596  00-0038128    M.Willis        Malik Willis       QB             QB   
5597  00-0038128    M.Willis        Malik Willis       QB             QB   

                                           headshot_url recent_team  season  \
0                                                  None         PIT    2010   
1                                                  None         PIT    2010   
2                                                  None         PIT    2010   
4                                                  None         JAX    2010   
5                                                  None         NYJ    2010   
...                                                 ...         ...     ...   
5593  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5594  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5595  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5596  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   
5597  https://static.www.nfl.com/image/private/f_aut...         TEN    2022   

      week season_type  ... receiving_first_downs  receiving_epa  \
0        2         REG  ...                   0.0            NaN   
1        3         REG  ...                   0.0            NaN   
2        4         REG  ...                   0.0            NaN   
4        7         REG  ...                   0.0            NaN   
5        4         REG  ...                   0.0            NaN   
...    ...         ...  ...                   ...            ...   
5593     9         REG  ...                   0.0            NaN   
5594    10         REG  ...                   0.0            NaN   
5595    13         REG  ...                   0.0            NaN   
5596    15         REG  ...                   0.0            NaN   
5597    16         REG  ...                   0.0            NaN   

      receiving_2pt_conversions  racr  target_share  air_yards_share  wopr  \
0                             0   NaN           NaN              NaN   NaN   
1                             0   NaN           NaN              NaN   NaN   
2                             0   NaN           NaN              NaN   NaN   
4                             0   NaN           NaN              NaN   NaN   
5                             0   NaN           NaN              NaN   NaN   
...                         ...   ...           ...              ...   ...   
5593                          0   NaN           NaN              NaN   NaN   
5594                          0   NaN           NaN              NaN   NaN   
5595                          0   NaN           NaN              NaN   NaN   
5596                          0   NaN           NaN              NaN   NaN   
5597                          0   NaN           NaN              NaN   NaN   

      special_teams_tds  fantasy_points  fantasy_points_ppr  
0                   0.0        1.000000            1.000000  
1                   0.0       18.040001           18.040001  
2                   0.0        4.040000            4.040000  
4                   0.0       13.680000           13.680000  
5                   0.0        0.280000            0.280000  
...                

In [35]:
draft = nfl.import_draft_picks(years)

In [36]:
draft

season  round  pick team     gsis_id pfr_player_id     cfb_player_id  \
8577     2010      1     1  STL  00-0027854      BradSa00    sam-bradford-1   
8578     2010      1     2  DET  00-0027855      SuhxNd99   ndamukong-suh-1   
8579     2010      1     3  TAM  00-0027856      McCoGe99    gerald-mccoy-1   
8580     2010      1     4  WAS        None      WillTr21  trent-williams-2   
8581     2010      1     5  KAN  00-0027858      BerrEr99      eric-berry-1   
...       ...    ...   ...  ...         ...           ...               ...   
11892    2017      7   249  SEA  00-0033594      CarsCh00    chris-carson-1   
11893    2017      7   250  DET  00-0033595      OConPa00     pat-oconnor-1   
11894    2017      7   251  CIN  00-0033798      SchrMa00   mason-schreck-1   
11895    2017      7   252  CLE  00-0033596      DayeMa00      matt-dayes-1   
11896    2017      7   253  DEN  00-0033597      KellCh00      chad-kelly-1   

      pfr_player_name    hof position  ... pass_ints rush_atts rush_yards  \
8577     Sam Bradford  False       QB  ...      61.0     146.0      340.0   
8578    Ndamukong Suh  False       DT  ...       0.0       0.0        0.0   
8579     Gerald McCoy  False       DT  ...       0.0       0.0        0.0   
8580   Trent Williams  False        T  ...       0.0       0.0        0.0   
8581       Eric Berry  False       DB  ...       0.0       0.0        0.0   
...               ...    ...      ...  ...       ...       ...        ...   
11892    Chris Carson  False       RB  ...       0.0     769.0     3502.0   
11893    Pat O'Connor  False       DE  ...       0.0       0.0        0.0   
11894   Mason Schreck  False       TE  ...       0.0       0.0        0.0   
11895   Matthew Dayes  False       RB  ...       0.0       5.0       13.0   
11896      Chad Kelly  False       QB  ...       0.0       1.0       -1.0   

       rush_tds  receptions  rec_yards  rec_tds  def_solo_tackles  def_ints  \
8577        2.0         1.0        5.0      0.0               NaN       NaN   
8578        0.0         0.0        0.0      0.0             392.0       1.0   
8579        0.0         0.0        0.0      0.0             235.0       NaN   
8580        0.0         0.0        0.0      0.0               1.0       NaN   
8581        0.0         0.0        0.0      0.0             377.0      14.0   
...         ...         ...        ...      ...               ...       ...   
11892      24.0       107.0      804.0      7.0               NaN       NaN   
11893       0.0         0.0        0.0      0.0              14.0       NaN   
11894       0.0         2.0        6.0      0.0               1.0       NaN   
11895       0.0         4.0       29.0      0.0               9.0       NaN   
11896       0.0         0.0        0.0      0.0               NaN       NaN   

      def_sacks  
8577        NaN  
8578       71.5  
8579       59.5  
8580        NaN  
8581        5.5  
...         ...  
11892       NaN  
11893       1.5  
11894       NaN  
11895       NaN  
11896       NaN  

[3320 rows x 36 columns]

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69669 entries, 0 to 5630
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    69669 non-null  object 
 1   player_name                  53133 non-null  object 
 2   player_display_name          69669 non-null  object 
 3   position                     69600 non-null  object 
 4   position_group               69600 non-null  object 
 5   headshot_url                 58874 non-null  object 
 6   recent_team                  69669 non-null  object 
 7   season                       69669 non-null  int32  
 8   week                         69669 non-null  int32  
 9   season_type                  69669 non-null  object 
 10  opponent_team                69669 non-null  object 
 11  completions                  69669 non-null  int32  
 12  attempts                     69669 non-null  int32  
 13  passing_yards         

In [34]:
data

player_id player_name player_display_name position position_group  \
0     00-0000865        None       Charlie Batch       QB             QB   
1     00-0000865        None       Charlie Batch       QB             QB   
2     00-0000865        None       Charlie Batch       QB             QB   
3     00-0001231        None        Josh Bidwell        P           SPEC   
4     00-0001545        None         Todd Bouman       QB             QB   
...          ...         ...                 ...      ...            ...   
5626  00-0038134    K.Walker      Kenneth Walker       RB             RB   
5627  00-0038134    K.Walker      Kenneth Walker       RB             RB   
5628  00-0038134    K.Walker      Kenneth Walker       RB             RB   
5629  00-0038147   D.Baldwin      Daylen Baldwin       WR             WR   
5630  00-0038149       J.Ali            Josh Ali       WR             WR   

                                           headshot_url recent_team  season  \
0                                                  None         PIT    2010   
1                                                  None         PIT    2010   
2                                                  None         PIT    2010   
3                                                  None         WAS    2010   
4                                                  None         JAX    2010   
...                                                 ...         ...     ...   
5626  https://static.www.nfl.com/image/private/f_aut...         SEA    2022   
5627  https://static.www.nfl.com/image/private/f_aut...         SEA    2022   
5628  https://static.www.nfl.com/image/private/f_aut...         SEA    2022   
5629  https://static.www.nfl.com/image/private/f_aut...         CLE    2022   
5630  https://static.www.nfl.com/image/private/f_aut...         ATL    2022   

      week season_type  ... receiving_first_downs  receiving_epa  \
0        2         REG  ...                   0.0            NaN   
1        3         REG  ...                   0.0            NaN   
2        4         REG  ...                   0.0            NaN   
3        1         REG  ...                   0.0            NaN   
4        7         REG  ...                   0.0            NaN   
...    ...         ...  ...                   ...            ...   
5626    17         REG  ...                   1.0       0.968106   
5627    18         REG  ...                   0.0       0.430252   
5628    19        POST  ...                   0.0      -0.309929   
5629    15         REG  ...                   2.0       1.791362   
5630    18         REG  ...                   0.0      -0.519020   

      receiving_2pt_conversions  racr  target_share  air_yards_share  \
0                             0   NaN           NaN              NaN   
1                             0   NaN           NaN              NaN   
2                             0   NaN           NaN              NaN   
3                             0   NaN           NaN              NaN   
4                             0   NaN           NaN              NaN   
...                         ...   ...           ...              ...   
5626                          0  -9.0      0.035714        -0.003717   
5627                          0 -10.0      0.035714        -0.003745   
5628                          0   0.0      0.029412         0.000000   
5629                          0   1.0      0.076923         0.223214   
5630                          0   0.0      0.035714         0.134731   

          wopr  special_teams_tds  fantasy_points  fantasy_points_ppr  
0          NaN                0.0        1.000000            1.000000  
1          NaN                0.0       18.040001           18.040001  
2          NaN                0.0        4.040000            4.040000  
3          NaN                0.0        0.000000            0.000000  
4          NaN                0.0       13.680000           13.680000  
...        ...                .

In [ ]:
df = data[data['player']]

In [30]:
data[data['player_display_name'] == 'Jared Goff']['season']

5202    2016
5203    2016
5204    2016
5205    2016
5206    2016
        ... 
1843    2021
1844    2021
1845    2021
1846    2021
1847    2021
Name: season, Length: 89, dtype: int32

In [29]:
nfl.import_draft_values()

pick  stuart  johnson     hill   otc    pff
0       1    34.6     3000  1000.00  3000  1.135
1       2    30.2     2600   717.17  2635  1.099
2       3    27.6     2200   514.33  2421  1.063
3       4    25.8     1800   490.52  2270  1.027
4       5    24.3     1700   467.81  2152  0.991
..    ...     ...      ...      ...   ...    ...
257   258     0.0        0     1.08    76  0.047
258   259     0.0        0     1.06    74  0.047
259   260     0.0        0     1.04    72  0.046
260   261     0.0        0     1.02    70  0.046
261   262     0.0        0     1.00    68  0.046

[262 rows x 6 columns]

In [15]:
wins = nfl.import_win_totals([2021, 2023])

In [17]:
wins[wins['game_id'] == '2021_01_DAL_TB']

game_id market_type   abbr  lines  odds  opening_lines  \
437011  2021_01_DAL_TB  money_line    DAL    NaN   378            0.0   
437012  2021_01_DAL_TB  money_line    DAL    NaN   378            0.0   
437013  2021_01_DAL_TB  money_line     TB    NaN  -425            0.0   
437014  2021_01_DAL_TB  money_line     TB    NaN  -425            0.0   
437015  2021_01_DAL_TB  money_line    DAL    NaN   350            0.0   
...                ...         ...    ...    ...   ...            ...   
437138  2021_01_DAL_TB       total   over   52.5  -107           51.5   
437139  2021_01_DAL_TB       total  under   52.5  -110           51.5   
437140  2021_01_DAL_TB       total   over   52.5  -110           51.5   
437141  2021_01_DAL_TB       total  under   52.0  -111           53.0   
437142  2021_01_DAL_TB       total   over   52.0  -111           53.0   

        opening_odds      book  season  
437011         240.0    5Dimes    2021  
437012         240.0  sportbet    2021  
437013        -280.0    5Dimes    2021  
437014        -280.0  sportbet    2021  
437015         230.0    bet365    2021  
...              ...       ...     ...  
437138        -116.0  BETMANIA    2021  
437139        -104.0   SKYBOOK    2021  
437140        -116.0   SKYBOOK    2021  
437141        -111.0  BetRegal    2021  
437142        -111.0  BetRegal    2021  

[132 rows x 9 columns]

In [16]:
wins['season']

game_id market_type   abbr  lines  odds  opening_lines  \
437011   2021_01_DAL_TB  money_line    DAL    NaN   378            0.0   
437012   2021_01_DAL_TB  money_line    DAL    NaN   378            0.0   
437013   2021_01_DAL_TB  money_line     TB    NaN  -425            0.0   
437014   2021_01_DAL_TB  money_line     TB    NaN  -425            0.0   
437015   2021_01_DAL_TB  money_line    DAL    NaN   350            0.0   
...                 ...         ...    ...    ...   ...            ...   
478076  2021_18_WAS_NYG       total   over   38.5  -114           39.0   
478077  2021_18_WAS_NYG       total  under   38.5  -103           39.0   
478078  2021_18_WAS_NYG       total   over   38.5  -117           39.0   
478079  2021_18_WAS_NYG       total  under   38.5  -111           38.5   
478080  2021_18_WAS_NYG       total   over   38.5  -111           38.5   

        opening_odds      book  season  
437011         240.0    5Dimes    2021  
437012         240.0  sportbet    2021  
437013        -280.0    5Dimes    2021  
437014        -280.0  sportbet    2021  
437015         230.0    bet365    2021  
...              ...       ...     ...  
478076        -115.0  BETMANIA    2021  
478077        -102.0   SKYBOOK    2021  
478078        -118.0   SKYBOOK    2021  
478079        -111.0  BetRegal    2021  
478080        -111.0  BetRegal    2021  

[34052 rows x 9 columns]

In [9]:
data

player_id   player_name player_display_name position position_group  \
0     00-0019596       T.Brady           Tom Brady       QB             QB   
1     00-0019596       T.Brady           Tom Brady       QB             QB   
2     00-0019596       T.Brady           Tom Brady       QB             QB   
3     00-0019596       T.Brady           Tom Brady       QB             QB   
4     00-0019596       T.Brady           Tom Brady       QB             QB   
...          ...           ...                 ...      ...            ...   
5648  00-0039165  Z.Charbonnet     Zach Charbonnet       RB             RB   
5649  00-0039165  Z.Charbonnet     Zach Charbonnet       RB             RB   
5650  00-0039165  Z.Charbonnet     Zach Charbonnet       RB             RB   
5651  00-0039165  Z.Charbonnet     Zach Charbonnet       RB             RB   
5652  00-0039165  Z.Charbonnet     Zach Charbonnet       RB             RB   

                                           headshot_url recent_team  season  \
0     https://static.www.nfl.com/image/private/f_aut...          TB    2022   
1     https://static.www.nfl.com/image/private/f_aut...          TB    2022   
2     https://static.www.nfl.com/image/private/f_aut...          TB    2022   
3     https://static.www.nfl.com/image/private/f_aut...          TB    2022   
4     https://static.www.nfl.com/image/private/f_aut...          TB    2022   
...                                                 ...         ...     ...   
5648  https://static.www.nfl.com/image/private/f_aut...         SEA    2023   
5649  https://static.www.nfl.com/image/private/f_aut...         SEA    2023   
5650  https://static.www.nfl.com/image/private/f_aut...         SEA    2023   
5651  https://static.www.nfl.com/image/private/f_aut...         SEA    2023   
5652  https://static.www.nfl.com/image/private/f_aut...         SEA    2023   

      week season_type  ... receiving_first_downs  receiving_epa  \
0        1         REG  ...                   0.0            NaN   
1        2         REG  ...                   0.0            NaN   
2        3         REG  ...                   0.0            NaN   
3        4         REG  ...                   0.0            NaN   
4        5         REG  ...                   0.0            NaN   
...    ...         ...  ...                   ...            ...   
5648    14         REG  ...                   0.0      -0.787724   
5649    15         REG  ...                   0.0      -0.442067   
5650    16         REG  ...                   0.0      -1.961893   
5651    17         REG  ...                   2.0       1.644468   
5652    18         REG  ...                   2.0       1.740490   

      receiving_2pt_conversions      racr  target_share  air_yards_share  \
0                             0       NaN           NaN              NaN   
1                             0       NaN           NaN              NaN   
2                             0       NaN           NaN              NaN   
3                             0       NaN           NaN              NaN   
4                             0       NaN           NaN              NaN   
...                         ...       ...           ...              ...   
5648                          0 -0.666667      0.032258        -0.025532   
5649                          0  0.000000      0.031250         0.012658   
5650                          0  0.307692      0.055556        -0.056034   
5651                          0  3.000000      0.151515         0.057778   
5652                          0  5.600000      0.076923         0.031447   

          wopr  special_teams_tds  fantasy_points  fantasy_points_ppr  
0          NaN                0.0           10.38               10.38  
1          NaN                0.0            9.40                9.40  
2          NaN                0.0           14.74               14.74  
3          NaN                0.0           25.40               25.40  
4          NaN                0

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)>